In [2]:
import numpy as np
import pandas as pd
%matplotlib inline
import matplotlib.pyplot as plt
from os.path import join, basename,exists
from glob import glob
from tqdm import tqdm
from infrasound import *

from obspy import read
from scipy.signal import correlate, correlation_lags, butter, filtfilt, spectrogram
from numpy.fft import fft, fftfreq, ifft, fftshift
from sklearn.preprocessing import minmax_scale
from datetime import timedelta

ac_calib = 8.2928e-05
name_dic = {'be4':'lower','a3m':'upper','ad8':'failed array'}
height_dic = {'lower-p0':0.33,'lower-p1':0.66,'lower-p2':1,'upper-p0':1.33,'upper-p1':np.nan,'upper-p2':2}

in_dir = '/home/zacharykeskinen/data/infrasound/infrasound/processed' #infrasound/banner/infrasound/processed/'
in_dir = '../../data/banner/infrasound/processed'
exists(in_dir)


# l = os.listdir(in_dir)
l = glob(os.path.join(in_dir, 'c0be4*'))
r = []
for i in l:
    i = os.path.basename(i)
    j = i[5:11]
    if j not in r:
        r.append(j)
r.sort()

wx_dir = '../../data/banner/wx/'
wx_fs = glob(join(wx_dir, '*STAND_MONTH*'))
df = pd.DataFrame()
for fp in wx_fs:
    df = df.append(pd.read_csv(fp, skiprows=3), ignore_index= True)
df.Date = pd.to_datetime(df.Date)
df = df.set_index(df.Date)
# df = df[df.index > pd.to_datetime('2021-11-01')]
df.loc[:, 'SWE_m'] = df['WTEQ.I-1 (in) ']*0.0254
df.loc[:, 'SD_m'] = df['SNWD.I-1 (in) ']*0.0254
df = df.drop_duplicates()